## Implementação do Modelo

### Importando bibliotecas

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go

### Importando dataset

In [3]:
dados = pd.read_csv('../dados/medidas_geral_e_defeitos.csv')
df = pd.DataFrame(dados)

### Definindo multiíndice

In [4]:
df = df.set_index(['Ordem de prod', 'Ref. do Artigo', 'Data de prod'])

In [5]:
df

Temperatura (F°)  Peso médio (g)  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03             2126.97           169.3   
                             2024-01-04             2118.92           169.7   
                             2024-01-05             2125.00           173.6   
                             2024-01-06             2125.40           170.2   
                             2024-01-07             2126.76           171.7   
...                                                     ...             ...   
194121        C0516.0000R    2024-06-26             2154.44           169.0   
                             2024-06-27             2152.32           168.9   
                             2024-06-28             2145.60           169.3   
                             2024-06-29             2145.60           168.8   
                             2024-06-30             2148.00           169.2   

                                           Comprimento médio (mm)  Diâmetro  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03                     110.4      32.5   
                             2024-01-04                      85.6      18.1   
                             2024-01-05                     140.5      32.9   
                             2024-01-06                     144.7      32.7   
                             2024-01-07                     141.5      30.2   
...                                                           ...       ...   
194121        C0516.0000R    2024-06-26                     124.3      32.6   
                             2024-06-27                     120.2      33.6   
                             2024-06-28                     124.1      34.2   
                             2024-06-29                     120.0      33.5   
                             2024-06-30                     122.9      34.6   

                                           Gotas cortadas (toneladas)  \
Ordem de prod Ref. do Artigo Data de prod                               
192970        C0516.0000R    2024-01-03                        20.948   
                             2024-01-04                        21.022   
                             2024-01-05                        21.449   
                             2024-01-06                        21.526   
                             2024-01-07                        21.439   
...                                                               ...   
194121        C0516.0000R    2024-06-26                        21.412   
                             2024-06-27                        21.447   
                             2024-06-28                        21.491   
                             2024-06-29                        21.419   
                             2024-06-30                        21.548   

                                           Total de Defeitos no Dia  \
Ordem de prod Ref. do Artigo Data de prod                             
192970        C0516.0000R    2024-01-03                         222   
                             2024-01-04                         249   
                             2024-01-05                         268   
                             2024-01-06                         242   
                             2024-01-07                         232   
...                                                             ...   
194121        C0516.0000R    2024-06-26                         258   
                             2024-06-27                         218   
                             2024-06-28                         218   
                             2024-06-29                         213   
                             2024-06-30                         256   

                                          Defeito Mais Frequente do Dia  \
Ordem de prod Ref. do Artigo Data de p

### Verificando se o dataframe está limpo e padronizado

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 200 entries, (192970, 'C0516.0000R', '2024-01-03') to (194121, 'C0516.0000R', '2024-06-30')
Columns: 134 entries, Temperatura (F°) to Crú
dtypes: float64(5), int64(128), object(1)
memory usage: 217.8+ KB


In [7]:
df.isnull().sum()

Temperatura (F°)              0
Peso médio (g)                0
Comprimento médio (mm)        0
Diâmetro                      0
Gotas cortadas (toneladas)    0
                             ..
Ferrugem no corpo             0
Fagulha                       0
Alça deformada                0
Instabilidade                 0
Crú                           0
Length: 134, dtype: int64

### Criando Dataframe sem índice

In [8]:
df_reset = df.reset_index()

### Variância dos defeitos por dia

In [9]:
fig = px.box(
    df_reset,
    y='Total de Defeitos no Dia',
    title='Distribuição de Total de Defeitos no Dia',
    points='all',
    color_discrete_sequence=['#636EFA']
)

# Ajustar layout
fig.update_layout(
    yaxis_title="Total de Defeitos no Dia",
    xaxis_title="",
    title_font=dict(size=18),
    font=dict(size=12),
    height=600,
    width=800,
    margin=dict(t=50, l=50, r=50, b=50)
)

# Exibir o gráfico
fig.show()

### Calculando total de frascos no dia

In [10]:
df_reset['Total de Frascos no Dia (unidade)'] = (df_reset['Gotas cortadas (toneladas)'] * 1000000) / df_reset['Peso médio (g)']
df_reset['Total de Frascos no Dia (unidade)'] = df_reset['Total de Frascos no Dia (unidade)'].astype(int)

In [11]:
df_reset.head(10)

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Gotas cortadas (toneladas),Total de Defeitos no Dia,Defeito Mais Frequente do Dia,...,Cascão cortante,Espessura abaixo do mínimo,Fundo deformado,Bolha na alça,Ferrugem no corpo,Fagulha,Alça deformada,Instabilidade,Crú,Total de Frascos no Dia (unidade)
0,192970,C0516.0000R,2024-01-03,2126.97,169.3,110.4,32.5,20.948,222,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,123733
1,192970,C0516.0000R,2024-01-04,2118.92,169.7,85.6,18.1,21.022,249,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,123877
2,192970,C0516.0000R,2024-01-05,2125.00,173.6,140.5,32.9,21.449,268,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,123554
3,192970,C0516.0000R,2024-01-06,2125.40,170.2,144.7,32.7,21.526,242,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,126474
4,192970,C0516.0000R,2024-01-07,2126.76,171.7,141.5,30.2,21.439,232,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,124863
5,192970,C0516.0000R,2024-01-08,2126.76,166.5,140.6,32.5,21.506,229,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,129165
6,192970,C0516.0000R,2024-01-09,2126.60,169.6,128.4,32.9,21.524,191,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,126910
7,192970,C0516.0000R,2024-01-10,2126.60,169.4,127.0,33.0,21.427,222,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,126487
8,192970,C0516.0000R,2024-01-11,2126.60,169.2,110.9,33.9,7.947,293,Parada de Máquina em Troca,...,0,0,0,0,0,0,0,0,0,46968
9,193112,C1164.0000R,2024-01-01,2189.40,168.4,78.2,41.6,21.890,159,Sujo de óleo,...,0,0,0,0,0,0,0,0,0,129988


In [12]:
df_medidas_totais_por_ordem = df_reset.groupby(['Ordem de prod', 'Ref. do Artigo']).agg({'Data de prod': 'nunique', 
                                        'Temperatura (F°)' : 'mean', 
                                        'Peso médio (g)' : 'mean', 
                                        'Comprimento médio (mm)' : 'mean', 
                                        'Diâmetro' : 'mean', 
                                        'Gotas cortadas (toneladas)' : 'sum', 
                                        'Total de Frascos no Dia (unidade)' : 'sum', 
                                        'Total de Defeitos no Dia':'sum', 
                                        'Sujo de óleo': 'sum',
                                        'Aderido': 'sum', 
                                        'Dobra': 'sum', 
                                        'Trinca por choque térmico': 'sum',
                                        'Ombro mal cheio': 'sum', 
                                        'Racho na terminação': 'sum',
                                        'Lascado em produção': 'sum',
                                        'Frio': 'sum'}).reset_index()

In [13]:
df_medidas_totais_por_ordem.rename(columns={'Data de prod': 'Duração da ordem', 'Total de Frascos no Dia (unidade)':'Total de Frascos', 'Total de Defeitos no Dia': 'Total de Defeitos', 'Diâmetro': 'Diâmetro (mm)'}, inplace=True)

In [14]:
df_medidas_totais_por_ordem.head(10).round(2).sort_values('Duração da ordem', ascending=False)

,Ordem de prod,Ref. do Artigo,Duração da ordem,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro (mm),Gotas cortadas (toneladas),Total de Frascos,Total de Defeitos,Sujo de óleo,Aderido,Dobra,Trinca por choque térmico,Ombro mal cheio,Racho na terminação,Lascado em produção,Frio
7,193342,C0516.0000R,20,2125.33,169.96,114.87,33.44,408.76,2405338,3685,1158,8,123,506,92,122,247,0
2,193164,C1280.0000R,12,2119.41,147.47,100.12,35.43,236.13,1600487,2661,851,2,43,129,0,220,33,19
4,193207,C1298.0000R,10,2139.29,170.87,98.23,36.56,136.92,801014,2130,754,0,27,87,0,79,45,4
0,192970,C0516.0000R,9,2125.51,169.91,125.51,30.97,178.79,1052031,2148,437,2,77,176,68,28,12,7
6,193209,C0509.0000R,7,2131.82,191.36,44.37,12.19,121.81,636428,1807,366,0,130,92,2,30,35,27
8,193368,C1137.0000R,7,2158.78,171.59,77.51,33.83,147.66,860417,1464,439,0,13,28,0,146,9,0
3,193206,C1284.0000R,4,2170.66,171.15,90.40,36.88,74.31,433842,1041,245,0,26,6,0,65,10,0
1,193112,C1164.0000R,3,2173.68,168.17,88.73,38.57,44.73,265727,584,259,0,35,22,0,9,2,0
9,193459,C0068.0000R,3,2148.19,190.83,96.67,37.57,37.59,197769,505,104,0,6,31,0,13,5,1
5,193208,C0104.0000R,2,2176.32,122.10,61.00,27.45,19.74,161756,369,48,2,4,17,0,3,2,0


In [15]:
df_medidas_totais_por_ordem['Ordem de prod'] = df_medidas_totais_por_ordem['Ordem de prod'].astype(str)

In [16]:
df_medidas_totais_por_ordem.columns

Index(['Ordem de prod', 'Ref. do Artigo', 'Duração da ordem',
       'Temperatura (F°)', 'Peso médio (g)', 'Comprimento médio (mm)',
       'Diâmetro (mm)', 'Gotas cortadas (toneladas)', 'Total de Frascos',
       'Total de Defeitos', 'Sujo de óleo', 'Aderido', 'Dobra',
       'Trinca por choque térmico', 'Ombro mal cheio', 'Racho na terminação',
       'Lascado em produção', 'Frio'],
      dtype='object')

In [17]:
df_medidas_totais_por_ordem.to_csv('../dados_modelo.csv', index=False)

### Relação Gotas Cortadas e Total de Defeitos

In [18]:
px.scatter(data_frame=df_reset, 
           x='Gotas cortadas (toneladas)', 
           y='Total de Defeitos no Dia', 
           color='Gotas cortadas (toneladas)', 
           color_continuous_scale='peach', 
           size='Total de Defeitos no Dia')

### Correlação das medidas com gotas cortadas

In [19]:
df_corr_medidas = df_reset[['Temperatura (F°)', 'Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro', 'Gotas cortadas (toneladas)', 'Sujo de óleo']]

In [20]:
correlacao_medidas = df_corr_medidas.corr()

In [21]:
fig = px.scatter_matrix(
    df_corr_medidas,
    dimensions=df_corr_medidas.columns,  # Dimensões a serem analisadas
    labels={col: col for col in df_corr_medidas.columns},  # Renomeando eixos se necessário
    title="Scatter Matrix das Medidas",
    height=800,
    width=800
)

# Mostrando o gráfico
fig.show()

# Solução - Prever o total de defeitos para nova ordem e total defeitos por dia

### Agrupar os dados por 'Ordem de prod' e 'Data de prod', somando os defeitos diários


In [22]:
df_sazonalidade = df_reset.groupby(['Ordem de prod', 'Data de prod'])[['Total de Defeitos no Dia', 'Gotas cortadas (toneladas)']].sum().reset_index()

### Converter a coluna 'Data de prod' para o tipo datetime

In [23]:
df_sazonalidade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Ordem de prod               200 non-null    int64  
 1   Data de prod                200 non-null    object 
 2   Total de Defeitos no Dia    200 non-null    int64  
 3   Gotas cortadas (toneladas)  200 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 6.4+ KB


In [24]:
df_sazonalidade['Data de prod'] = pd.to_datetime(df_sazonalidade['Data de prod'])

### Criar novas colunas com informações sobre a data

In [25]:
df_sazonalidade['Dia da Semana'] = df_sazonalidade['Data de prod'].dt.day_name()  # Nome do dia da semana
df_sazonalidade['Mês'] = df_sazonalidade['Data de prod'].dt.month  # Número do mês
df_sazonalidade['Dia do Ano'] = df_sazonalidade['Data de prod'].dt.dayofyear  # Número do dia do ano

In [26]:
df_sazonalidade

,Ordem de prod,Data de prod,Total de Defeitos no Dia,Gotas cortadas (toneladas),Dia da Semana,Mês,Dia do Ano
0,192970,2024-01-03,222,20.948,Wednesday,1,3
1,192970,2024-01-04,249,21.022,Thursday,1,4
2,192970,2024-01-05,268,21.449,Friday,1,5
3,192970,2024-01-06,242,21.526,Saturday,1,6
4,192970,2024-01-07,232,21.439,Sunday,1,7
...,...,...,...,...,...,...,...
195,194121,2024-06-26,258,21.412,Wednesday,6,178
196,194121,2024-06-27,218,21.447,Thursday,6,179
197,194121,2024-06-28,218,21.491,Friday,6,180
198,194121,2024-06-29,213,21.419,Saturday,6,181


### Calcular a duração de produção (número de dias únicos) por 'Ordem de prod'

In [27]:
duracao_por_ordem = df_sazonalidade.groupby('Ordem de prod')['Data de prod'].nunique().reset_index()

In [28]:
duracao_por_ordem

,Ordem de prod,Data de prod
0,192970,9
1,193112,3
2,193164,12
3,193206,4
4,193207,10
5,193208,2
6,193209,7
7,193342,20
8,193368,7
9,193459,3


### Renomear a coluna resultante para 'Dias de Duração'

In [29]:
duracao_por_ordem.columns = ['Ordem de prod', 'Dias de Duração']

### Agrupar os dados por 'Ordem de prod' e somar os defeitos diários

In [30]:
defeitos_por_ordem = df_sazonalidade.groupby('Ordem de prod')[['Total de Defeitos no Dia', 'Gotas cortadas (toneladas)']].sum().reset_index()

In [31]:
defeitos_por_ordem.head()

,Ordem de prod,Total de Defeitos no Dia,Gotas cortadas (toneladas)
0,192970,2148,178.788
1,193112,584,44.732
2,193164,2661,236.126
3,193206,1041,74.312
4,193207,2130,136.915


### Renomear a coluna resultante para 'Total de Defeitos por Ordem'

In [32]:
defeitos_por_ordem.columns = ['Ordem de prod', 'Total de Defeitos por Ordem', 'Gotas cortadas (toneladas)']

### Juntar as informações de 'Duracao por Ordem' e 'Defeitos por Ordem' em um único DataFrame

In [33]:
duracao_por_ordem = duracao_por_ordem.merge(defeitos_por_ordem, on='Ordem de prod', how='left')

In [34]:
duracao_por_ordem

,Ordem de prod,Dias de Duração,Total de Defeitos por Ordem,Gotas cortadas (toneladas)
0,192970,9,2148,178.788
1,193112,3,584,44.732
2,193164,12,2661,236.126
3,193206,4,1041,74.312
4,193207,10,2130,136.915
5,193208,2,369,19.743
6,193209,7,1807,121.813
7,193342,20,3685,408.755
8,193368,7,1464,147.662
9,193459,3,505,37.588


### Gerar um gráfico de dispersão (scatter) com as informações de duração e defeitos

In [35]:
px.scatter(data_frame=duracao_por_ordem, 
           x='Dias de Duração', 
           y='Total de Defeitos por Ordem', 
           color='Total de Defeitos por Ordem', 
           color_continuous_scale='peach', 
           size='Total de Defeitos por Ordem')

### Adicionar as informações de 'Duração por Ordem' no DataFrame original 'df_sazonalidade'

In [36]:
df_sazonalidade = df_sazonalidade.merge(duracao_por_ordem, on='Ordem de prod', how='left')

In [37]:
# Exibir o DataFrame final
df_sazonalidade

,Ordem de prod,Data de prod,Total de Defeitos no Dia,Gotas cortadas (toneladas)_x,Dia da Semana,Mês,Dia do Ano,Dias de Duração,Total de Defeitos por Ordem,Gotas cortadas (toneladas)_y
0,192970,2024-01-03,222,20.948,Wednesday,1,3,9,2148,178.788
1,192970,2024-01-04,249,21.022,Thursday,1,4,9,2148,178.788
2,192970,2024-01-05,268,21.449,Friday,1,5,9,2148,178.788
3,192970,2024-01-06,242,21.526,Saturday,1,6,9,2148,178.788
4,192970,2024-01-07,232,21.439,Sunday,1,7,9,2148,178.788
...,...,...,...,...,...,...,...,...,...,...
195,194121,2024-06-26,258,21.412,Wednesday,6,178,6,1436,127.025
196,194121,2024-06-27,218,21.447,Thursday,6,179,6,1436,127.025
197,194121,2024-06-28,218,21.491,Friday,6,180,6,1436,127.025
198,194121,2024-06-29,213,21.419,Saturday,6,181,6,1436,127.025


### Criando Dataframe agrupado por ordem de produção, com operações diferentes

In [38]:
defeitos_por_duracao = df_sazonalidade.groupby('Ordem de prod').agg({'Total de Defeitos no Dia': 'sum', 'Dias de Duração': 'first'})

### Renomear a coluna 'Total de Defeitos no Dia' para 'Total de Defeitos por Ordem'

In [39]:
defeitos_por_duracao.rename(columns={'Total de Defeitos no Dia': 'Total de Defeitos por Ordem'}, inplace=True)

In [40]:
defeitos_por_duracao

,Total de Defeitos por Ordem,Dias de Duração
Ordem de prod,,
192970,2148,9
193112,584,3
193164,2661,12
193206,1041,4
193207,2130,10
193208,369,2
193209,1807,7
193342,3685,20
193368,1464,7


## Média de defeitos por dias da semana

### Agrupando dados por 'Dia da Semana' e calculando médias

In [41]:
df_medias_dia_semana = df_sazonalidade.groupby('Dia da Semana')[['Total de Defeitos no Dia']].mean().sort_values('Total de Defeitos no Dia', ascending=False).round(2)

In [42]:
fig = px.bar(df_medias_dia_semana.reset_index(), 
             x='Dia da Semana', y='Total de Defeitos no Dia',
             text='Total de Defeitos no Dia',
             color='Total de Defeitos no Dia',
             color_continuous_scale='peach')

fig.update_traces(textposition='outside', texttemplate='%{text}', 
                  textfont_size=14, textfont_color='black')

fig.show()

## Média de defeitos por dia de produção

### Calcular a duração total de cada ordem

In [43]:
duracao_por_ordem = df_sazonalidade.groupby('Ordem de prod')['Data de prod'].nunique().reset_index()
duracao_por_ordem.columns = ['Ordem de prod', 'Dias de Duração']

### Merge com o DataFrame original

In [44]:
df_sazonalidade = df_sazonalidade.merge(duracao_por_ordem, on='Ordem de prod', how='left')

### Remover a coluna duplicada (se já havia 'Dias de Duração' no DataFrame original)

In [45]:
df_sazonalidade = df_sazonalidade.drop(columns=['Dias de Duração_x'])

### Renomear a coluna restante, se necessário

In [46]:
df_sazonalidade = df_sazonalidade.rename(columns={'Dias de Duração_y': 'Dias de Duração'})

### Padronizando a duração das ordens

Calcular o Dia Contado por Ordem

In [47]:
df_sazonalidade['Dia Contado'] = df_sazonalidade.groupby('Ordem de prod').cumcount() + 1

`.cumcount()`: Gera uma contagem cumulativa para cada linha dentro de cada grupo. A contagem começa em 0 para a primeira linha de cada grupo, 1 para a segunda linha, e assim por diante.


`+ 1`: Incrementa a contagem para começar em 1 em vez de 0 (como o "primeiro dia" de produção em vez de "dia 0").

In [48]:
df_sazonalidade[['Ordem de prod', 'Data de prod', 'Dia Contado']]

,Ordem de prod,Data de prod,Dia Contado
0,192970,2024-01-03,1
1,192970,2024-01-04,2
2,192970,2024-01-05,3
3,192970,2024-01-06,4
4,192970,2024-01-07,5
...,...,...,...
195,194121,2024-06-26,2
196,194121,2024-06-27,3
197,194121,2024-06-28,4
198,194121,2024-06-29,5


Calcular o Dia Normalizado

In [49]:
df_sazonalidade['Dia Normalizado'] = df_sazonalidade['Dia Contado'] / df_sazonalidade['Dias de Duração']

`Dia Contado`: Representa o dia específico dentro de uma ordem de produção (e.g., dia 1, dia 2...).

`Dias de Duração`: É o número total de dias necessários para completar a ordem de produção.

`Dia Contado `/ `Dias de Duração`: Faz com que cada dia seja representado como uma fração do ciclo total de produção.

Visualização

In [50]:
df_sazonalidade[['Ordem de prod', 'Dia Contado', 'Dias de Duração', 'Dia Normalizado', 'Total de Defeitos no Dia']]

,Ordem de prod,Dia Contado,Dias de Duração,Dia Normalizado,Total de Defeitos no Dia
0,192970,1,9,0.111111,222
1,192970,2,9,0.222222,249
2,192970,3,9,0.333333,268
3,192970,4,9,0.444444,242
4,192970,5,9,0.555556,232
...,...,...,...,...,...
195,194121,2,6,0.333333,258
196,194121,3,6,0.500000,218
197,194121,4,6,0.666667,218
198,194121,5,6,0.833333,213


- Isso é útil para comparar dados de ordens de diferentes durações. Sem essa normalização, ordens mais curtas ou mais longas não seriam diretamente comparáveis.

- O resultado da coluna 'Dia Normalizado' estará sempre entre 0 e 1, onde:

`0`: Representa o início da produção (primeiro dia).

`1`: Representa o final da produção (último dia).

Valores intermediários (e.g., 0.5) indicam a posição relativa no ciclo.

Exportando dataframe

In [51]:
df_sazonalidade_dias_ordem  = df_sazonalidade[['Ordem de prod', 'Dia Contado', 'Dias de Duração', 'Dia Normalizado', 'Total de Defeitos no Dia']]
df_sazonalidade_dias_ordem.to_csv('../dados_dias_modelo.csv', index=False)

### Calcular a média de defeitos por Dia Normalizado

In [52]:
defeitos_por_dia_normalizado = df_sazonalidade.groupby('Dia Normalizado')[['Total de Defeitos no Dia']].mean().reset_index()
defeitos_por_dia_normalizado.columns = ['Dia Normalizado', 'Média de Defeitos']

Visualização

In [53]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=defeitos_por_dia_normalizado['Dia Normalizado'],
                         y=defeitos_por_dia_normalizado['Média de Defeitos'],
                         mode='lines+markers', 
                         marker=dict(size=10),
                         line=dict(color='red'),
                         name='Média de Defeitos'))

fig.update_layout(
    title='Média de Defeitos por Dia Normalizado',
    title_font_size=16,
    xaxis_title='Dia Normalizado',
    xaxis_title_font_size=14,
    yaxis_title='Média de Defeitos',
    yaxis_title_font_size=14
)

fig.show()

### Calcular a média de defeitos por Dia Contado

Agrupar pelos valores inteiros de Dia Contado

In [54]:
defeitos_por_dia_contado = df_sazonalidade.groupby('Dia Contado')[['Total de Defeitos no Dia']].mean().reset_index()
defeitos_por_dia_contado.columns = ['Dia Contado', 'Média de Defeitos']

Visualização

In [55]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=defeitos_por_dia_contado['Dia Contado'],
                         y=defeitos_por_dia_contado['Média de Defeitos'],
                         mode='lines+markers', 
                         marker=dict(size=10),
                         line=dict(color='red'),
                         name='Média de Defeitos'))

fig.update_layout(
    title='Média de Defeitos por Dia Contado',
    title_font_size=16,
    xaxis_title='Dia Contado',
    xaxis_title_font_size=14,
    yaxis_title='Média de Defeitos',
    yaxis_title_font_size=14
)

fig.show()

## Implementação do Modelo de Regressão

### Resetando index

In [56]:
df_total_duracao_reset = defeitos_por_duracao.reset_index()

In [57]:
df_total_duracao_reset.head()

,Ordem de prod,Total de Defeitos por Ordem,Dias de Duração
0,192970,2148,9
1,193112,584,3
2,193164,2661,12
3,193206,1041,4
4,193207,2130,10


### Passo 1: Preparar os dados para o total de defeitos e a duração da ordem

In [58]:
df_combined = df_reset.merge(df_total_duracao_reset[['Ordem de prod', 'Total de Defeitos por Ordem', 'Dias de Duração']], on='Ordem de prod', how='left')

### Passo 2: Separar variáveis independentes (medidas e duração) e dependente (total de defeitos)

In [59]:
X = df_combined[['Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro', 'Dias de Duração']]
y = df_combined['Total de Defeitos por Ordem']

### Passo 3: Dividir os dados em treino e teste

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Passo 4: Normalizar os dados

In [61]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Passo 5: Treinar o modelo para prever o total de defeitos

In [62]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train_scaled, y_train)

GradientBoostingRegressor(random_state=42)

### Passo 6: Previsões de total de defeitos

In [63]:
y_pred_gb = gb_model.predict(X_test_scaled)

### Passo 7: Avaliar a performance do modelo

In [64]:
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print(f"Gradient Boosting - Mean Squared Error (MSE): {mse_gb}")
print(f"Gradient Boosting - R² Score: {r2_gb}")

Gradient Boosting - Mean Squared Error (MSE): 7698.757546270441
Gradient Boosting - R² Score: 0.9918282458806885


- **Mean Squared Error (MSE)**: Mede o erro médio ao quadrado entre os valores previstos pelo modelo e os valores reais. No caso, um MSE de 7698,76 indica que, em média, o modelo apresenta esse nível de erro nas previsões, o que pode ser considerado aceitável dependendo da escala dos defeitos totais.


- **R² Score**: Representa o quanto do comportamento dos dados o modelo consegue explicar, com valores próximos de 1,0 indicando alta precisão. Aqui, o R² de 0,99 mostra que o modelo explica 99% da variação nos defeitos totais, o que reflete um excelente ajuste aos dados.

> Meu modelo de Gradient Boosting está com um R² de 0,99, o que indica alta precisão nas previsões, embora haja sinais de ***overfitting*** devido ao impacto significativo da ***duração*** da ordem nos defeitos ***totais***. Apesar disso, o diferencial da solução está na capacidade de ***distribuir*** os defeitos totais previstos ao longo dos dias da nova ordem, permitindo identificar os dias críticos e agir preventivamente para otimizar a produção. 

## Testando modelo

### Criando nova ordem e prevendo com o modelo

In [65]:
nova_ordem = {
    'Peso médio (g)': 160,
    'Comprimento médio (mm)': 120,
    'Diâmetro': 30.2,
    'Dias de Duração': 15
}

X_nova_ordem = scaler.transform(pd.DataFrame([nova_ordem])) 
total_defeitos_previsto = gb_model.predict(X_nova_ordem)
print(f"Total de defeitos previsto para a nova ordem: {total_defeitos_previsto[0]}")

Total de defeitos previsto para a nova ordem: 2331.3429814026454


### Calcular a soma total dos defeitos para garantir que a soma seja igual ao total esperado

In [66]:
total_medio_defeitos = defeitos_por_dia_contado['Média de Defeitos'].sum()

- O que faz: Calcula a soma total da coluna 'Média de Defeitos', ou seja, o valor agregado da média de defeitos registrados por dia.

- Por que isso é útil: Este valor será usado para determinar a contribuição proporcional de cada dia na previsão do total de defeitos.

### Garantir que a soma dos defeitos seja proporcionalmente ajustada ao total de defeitos previsto

In [67]:
defeitos_por_dia_contado['Defeitos Previstos'] = (defeitos_por_dia_contado['Média de Defeitos'] / total_medio_defeitos) * total_defeitos_previsto

- O que faz: Estima a quantidade de defeitos previstos para cada dia com base na média de defeitos já registrada e o valor total de defeitos previstos.

**Passo a passo do cálculo**:

1.  `defeitos_por_dia_contado['Média de Defeitos']` / `total_medio_defeitos`: Calcula a proporção de defeitos de cada dia em relação ao total médio de defeitos.

2. Multiplica essa proporção pelo `total_defeitos_previsto` para distribuir os defeitos previstos entre os dias, proporcionalmente à média de defeitos observada.

- Por que isso é útil: Permite projetar o total de defeitos em cada dia com base em dados históricos.

### Recalcular a distribuição dos defeitos considerando o número de dias de duração

In [68]:
dias_duracao = nova_ordem['Dias de Duração']

### Selecionar as previsões apenas para os dias da nova ordem

In [69]:
df_dias_duracao = defeitos_por_dia_contado.head(dias_duracao).copy()  # Usando .copy() para evitar aviso

### Garantir que a soma dos defeitos nos dias da nova ordem seja igual ao total de defeitos previsto


Ajustar a distribuição proporcionalmente para que a soma dos defeitos seja igual ao `total_defeitos_previsto`

In [70]:
soma_defeitos_dia = df_dias_duracao['Defeitos Previstos'].sum()
ajuste_fator = total_defeitos_previsto / soma_defeitos_dia

# Ajustar os defeitos previstos diretamente no DataFrame com .loc[]
df_dias_duracao.loc[:, 'Defeitos Previstos'] = df_dias_duracao['Defeitos Previstos'] * ajuste_fator

- O que faz:

`soma_defeitos_dia`: Soma os valores da coluna 'Defeitos Previstos' no DataFrame `df_dias_duracao`.

`ajuste_fator`: Calcula o fator necessário para ajustar a soma total dos defeitos previstos ao valor exato de `total_defeitos_previsto`.

Multiplica cada valor da coluna 'Defeitos Previstos' pelo `ajuste_fator` para garantir que a soma total da coluna corresponda ao valor exato de `total_defeitos_previsto`.

- Por que isso é útil: Durante o cálculo inicial de `'Defeitos Previstos'`, pode haver pequenas discrepâncias de arredondamento que fazem com que a soma dos valores não corresponda exatamente ao `total_defeitos_previsto`. Esse ajuste corrige isso.

### Visualização Geral

In [71]:
print(f"Total de defeitos previsto para a nova ordem: {total_defeitos_previsto[0]:.2f}")

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_dias_duracao['Dia Contado'],
                         y=df_dias_duracao['Defeitos Previstos'],
                         mode='lines+markers',
                         marker=dict(size=10),
                         line=dict(color='green'),
                         name='Defeitos Previstos'))

fig.update_layout(
    title=f'Previsão de Defeitos ao Longo dos {dias_duracao} Dias da Nova Ordem',
    title_font_size=16,
    xaxis_title='Dia de Produção',
    xaxis_title_font_size=14,
    yaxis_title='Defeitos Previstos',
    yaxis_title_font_size=14
)

fig.show()

df_dias_duracao[['Dia Contado', 'Defeitos Previstos']].set_index('Dia Contado')

Total de defeitos previsto para a nova ordem: 2331.34


,Defeitos Previstos
Dia Contado,
1,172.432072
2,189.648274
3,166.055019
4,170.549168
5,166.337233
6,164.986433
7,144.865521
8,155.170861
9,165.541841
